# Data Processing Walkthrough
This is the notebook used to detail the data processing for training the Vocal Pitch Modulator.

The goes through in detail (with plots and prints) how the data is organized. 

## Global variables/Imports
Run these cells before running either of the following sections.

In [ ]:
%load_ext autoreload
%autoreload 1

import os
import csv

import scipy.io as sio
from scipy.io import wavfile
from scipy.io.wavfile import write

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots

%aimport VPM
from VPM import *
%aimport Utils
from Utils import *

In [ ]:
# Constants that should not change without the dataset being changed
n_pitches = 16
n_vowels = 12
n_people = 3

# These dictionaries are more for reference than anything
label_to_vowel = { 0: "bed",  1: "bird",   2: "boat",  3: "book", 
                   4: "cat",  5: "dog",    6: "feet",  7: "law",  
                   8: "moo",  9: "nut",   10: "pig",  11: "say" }

vowel_to_label = { "bed": 0,  "bird": 1,  "boat":  2, "book":  3,
                   "cat": 4,  "dog":  5,  "feet":  6, "law":   7,
                   "moo": 8,  "nut":  9,  "pig":  10, "say":  11}

noteidx_to_pitch = {  0: "A2",   1: "Bb2",  2: "B2",   3: "C3",
                      4: "Db3",  5: "D3",   6: "Eb3",  7: "E3", 
                      8: "F3",   9: "Gb3", 10: "G3",  11: "Ab3",
                     12: "A3",  13: "Bb3", 14: "B3",  15: "C4" }

## Getting data references
Read the reference csv to relevant data structure.

`data_ref_list` is the list of filenames in the dataset in a 3d array format.
A specific file is accessed with `data_ref_list[vowel_idx][pitch_idx][person_idx]`.

`flat_data_ref_list` is the list of filenames in the dataset as a 1d array. To access a specific file, use `flat_data_ref_list[flat_ref_idx(vowel, pitch, person)]`

In [ ]:
# e.g. data_list[vowel_to_label["dog"]][5][1]
data_ref_list = create_data_ref_list(os.path.join("Data", 'dataset_files.csv'),
                                     n_pitches, n_vowels, n_people)
# print(data_ref_list)
# e.g. flat_data_ref_list[flat_ref_idx(3, 1, 2)]
flat_data_ref_list = flatten_3d_array(data_ref_list, 
                                      n_vowels, n_pitches, n_people)

The following are the accessor functions used to compute indices from flat to 3d and vice versa.

`flat_ref_idx` returns a `flat_ref_idx`, given a `(vowel, pitch, person)`, while `nd_ref_idx` returns `vowel, pitch, person`, given a `flat_ref_idx`.

In [ ]:
# Returns a flat_ref_idx, given a vowel, pitch, person
flat_ref_idx = lambda vowel, pitch, person: flat_3d_array_idx(
    vowel, pitch, person, n_vowels, n_pitches, n_people)
# Returns vowel, pitch, person, given a flat_ref_idx
nd_ref_idx = lambda idx: nd_array_idx(idx, n_vowels, n_pitches, n_people)

In [ ]:
print("Data ref list ({}):".format(len(flat_data_ref_list)), 
      flat_data_ref_list)

## Data-label Pitch Index pairs
Generate the data-label pitch index pairs. This is an array where each element is a 3-tuple of `[shift_amt, input_pitch_idx, label_pitch_idx]`. `data_label_pairs` stores all these 3-tuples in a single 2d array, while `data_label_dict` stores them in a dictionary, indexed by `shift_amt`.


In [ ]:
data_label_pairs, data_label_pairs_dict = create_data_label_pairs(n_pitches)

In [ ]:
print("Total data-label pairs ({}):".format(len(data_label_pairs)), 
      data_label_pairs)

## Get All .wav Data
Get the wav file data into a single matrix, where each element `all_wav_data[idx]` is the wavfile content of the file at `flat_data_ref_list[idx]`. To retrieve the 3d indices of a specific index, use `vowel, pitch, person = nd_ref_idx(idx)`, and to retrieve the index in the `flat_data_ref_list` for a specific vowel, pitch and person, use `flat_data_ref_list[flat_ref_idx(vowel, pitch, person)]`.


In [ ]:
all_wav_data = load_wav_files(os.path.join("Data", "dataset"), 
                              flat_data_ref_list)

In [ ]:
print("All wav data length: {}\nTrack length: {}".format(
      all_wav_data.shape, all_wav_data[0].shape))

## Create all spectrograms
Get the spectrograms for each wav in `all_wav_data`. The spectrogram at `all_spectrograms[idx]` is the spectrogram of the wav at `all_wav_data[idx]`.

Note that to access a specific fft for a window, you need to use the indexing `all_spectrograms[file_idx][:, window]`, since the shape is `(n_wavs, n_freq_bins, n_windows)`.

In [ ]:
all_spectrograms = np.array([ stft(waveform, plot=False) 
                              for waveform in all_wav_data ])

In [ ]:
print("All spectrograms has shape: {} (n_wavs, n_freq_bins, n_windows)\n"
      .format(all_spectrograms.shape))

print("FFT Spectrogram of vowel 4, pitch 3, person 2 ({}):"
      .format(flat_data_ref_list[flat_ref_idx(4, 3, 2)]))
plot_ffts_spectrogram(all_spectrograms[flat_ref_idx(4, 3, 2)], sample_rate,
                      flat_data_ref_list[flat_ref_idx(4, 3, 2)])

## Create Mel Spectrograms and MFCC
Get the mel spectrograms/MFCC for each ffts (spectrogram) in `all_spectrograms` (similar indexing as above).

In [ ]:
all_mels, all_mfcc = map(np.array, map(list, zip(*
                         [ ffts_to_mel(ffts, n_mels = 128) 
                           for ffts in all_spectrograms ])))

In [ ]:
print("All mels has shape: {} (n_wavs, n_mels, n_windows)"
      .format(all_mels.shape))
print("All mfccs has shape: {} (n_wavs, n_mfcc, n_windows)\n"
      .format(all_mfcc.shape))

print("Mel Spectrogram of vowel 4, pitch 3, person 2 ({}):"
      .format(flat_data_ref_list[flat_ref_idx(4, 3, 2)]))
plot_mel_spectrogram(all_mels[flat_ref_idx(4, 3, 2)], sample_rate,
                     flat_data_ref_list[flat_ref_idx(4, 3, 2)])
print("MFCC of vowel 4, pitch 3, person 2 ({}):"
      .format(flat_data_ref_list[flat_ref_idx(4, 3, 2)]))
plot_mfcc(all_mfcc[flat_ref_idx(4, 3, 2)], sample_rate,
          flat_data_ref_list[flat_ref_idx(4, 3, 2)])

## Data/Label Example 1: Flattening our 3d arrays to 2d arrays for Autoencoder Input
For this example, we look at how we can flatten our 3d arrays to 2d arrays. In this case, we are not doing any data-label pairing. This is useful for e.g. Autoencoder training, where the data *is* the label (we try to recreate the data).

`all_wav_data` is a 2d array, but `all_spectrograms (576, 513, 58)`, `all_mels (576, 128, 58)` and `all_mfcc (576, 20, 58)` are 3d, where indexed by `[file_idx, bin_idx, window_idx]`, where `file_idx` is `flat_ref_idx(vowel, pitch, person)`, `bin_idx` is frequency bin for spectrograms/mel bin for mels/mfcc number for mfccs, and `window_idx` is the window number as a result of windowing the wav data.

To make these into 2d arrays for input into the neural networks, we can do something like so. This example flattens the `all_mfcc` array. Pay attention to the fact that when accessing `all_mfcc`, as mentioned before, we use `all_mfcc[wav_file_idx][:, window_idx]`.

In [ ]:
# Retrieve the shape first
n_files, _, n_windows = all_mfcc.shape

# Returns a flat 2d idx, given a wavfile index and a window index, where
# wavfile index can be retrieved from `flat_ref_idx(vowel, pitch, person)`.
flat_data_idx = lambda wav_idx, win_idx: flat_2d_array_idx(
    wav_idx, win_idx, n_files, n_windows)

# data:   (33408, 20) (n_wavs * n_windows, n_mfcc)
data_prenorm = np.array([ all_mfcc[wav_file_idx][:, window_idx] 
                          for wav_file_idx in range(n_files) 
                          for window_idx in range(n_windows) ])

In [ ]:
print("data_prenorm has shape: {} (n_wavs * n_windows, n_mfcc)"
      .format(data_prenorm.shape),
      "\nWhereas all_mfcc has shape: {} (n_wavs, n_mfcc, n_windows)\n"
      .format(all_mfcc.shape))

print("MFCC of vowel 4, pitch 3, person 2, window 12 from `all_mfcc`\n({}):"
      .format(flat_data_ref_list[flat_ref_idx(4, 3, 2)]))
print(all_mfcc[flat_ref_idx(4, 3, 2)].T[12])
print("The same MFCC accessed from the flattened array:")
print(data_prenorm[flat_data_idx(flat_ref_idx(4, 3, 2), 12)])

### Normalization/Denormalization
When dealing with NN, the NN outputs from `[0, 1]`. Hence, we want to normalize our inputs, and denormalize our outputs (e.g. after passing it through our model). This can be done as shown - `data` is the normalized `data_prenorm`.

In [ ]:
data, scales = normalize_rows(data_prenorm, NormMode.REAL_TO_ZERO_ONE)

In [ ]:
print("Normalized MFCC of vowel 4, pitch 3, person 2, window 12:")
print(data[flat_data_idx(flat_ref_idx(4, 3, 2), 12)])

print("\nDenormalized MFCC:")
print(denormalize_rows(data, DenormMode.ZERO_ONE_TO_REAL, scales)
      [flat_data_idx(flat_ref_idx(4, 3, 2), 12)])

## Data/Label Example 2: Vowel Classifier Input
In this example, we do the same 3d to 2d array flattening as Example 1, but we are doing some data-label pairing - we want to align each file to its corresponding **vowel as its label**.

To get the vowel labels, we use `nd_ref_idx`. The `map(map(zip*)))` is a bit of python magic to extract a three lists from a list of 3-tuple (not too important), the main point is to illustrate how we get data-label pairings.

In [ ]:
n_files, _, n_windows = all_mfcc.shape

# vowel_labels: (576) (n_wavs)
all_vowel_labels, _, _ = map(np.array, map(list, zip(*
                         [ nd_ref_idx(idx) 
                           for idx in range(len(flat_data_ref_list)) ])))
# Returns a flat 2d idx, given a wavfile index and a window index
flat_data_idx = lambda wav_idx, win_idx: flat_2d_array_idx(
    wav_idx, win_idx, n_files, n_windows)

# data:   (33408, 20) (n_wavs * n_windows, n_mfcc)
# labels: (33408) (n_wavs * n_windows)
data_prenorm = np.array([ all_mfcc[wav_file_idx][:, window_idx] 
                          for wav_file_idx in range(n_files) 
                          for window_idx in range(n_windows) ])
labels = np.array([ all_vowel_labels[wav_file_idx]
                    for wav_file_idx in range(n_files)
                    for window_idx in range(n_windows) ])
# Perform normalization similar to example 1
data, scales = normalize_rows(data_prenorm, NormMode.REAL_TO_ZERO_ONE)

In [ ]:
print("Data of vowel 4, pitch 3, person 2, window 12 from `all_mfcc`\n({}):"
      .format(flat_data_ref_list[flat_ref_idx(4, 3, 2)]))
print(data[flat_data_idx(flat_ref_idx(4, 3, 2), 12)])
print("Label of vowel 4, pitch 3, person 2, window 12 should be 'cat' ({}), and it is: {}"
      .format(vowel_to_label["cat"],
              labels[flat_data_idx(flat_ref_idx(4, 3, 2), 12)]))

## Data/Label Example 3: Pitch Shifter Input
This is a more complicated example, which is the main meat of our program. In this example, we show how to pair up data-label pairs, where data is a wav file + the pitch shifted wav file, and the label is a *different* wav file, the wav file corresponding to the correct pitch of the shifted wav file.

We make use of the `data_labels_pairs_dict` to filter out combinations that are not possible. By running through all the possible combinations, we get a data-label set. 

Another thing we do is to make one person's wav file the validation set, while the other 2 comprise the training set.

In [ ]:
# X_train_base, Y_train: (_,513), (_,513)
# X_val_base, Y_val:     (_,513), (_,513)
X_train_base = []; X_val_base = []; Y_train = []; Y_val = [];
X_train_shifted = []; X_val_shifted = [];
list_of_test_idx = []

# Gather the shift_amt's from -2 to 2
for shift_amt in [ -2, -1, 0, 1, 2 ]:
    # Get the filter for wav files
    pairs = data_label_pairs_dict[shift_amt]
    # Iterate through vowel and pitches 
    for vow_idx in range(n_vowels):
        for pit_idx in range(n_pitches):
            # If the pair is valid, then proceed.
            if [shift_amt, pit_idx, pit_idx + shift_amt] in pairs:
                # Choose the person for this pitch/vowel to be used as test data.
                test_pid = int(np.random.rand() * 3)
                # With vowel, pitch and person ID, we can get a wav_idx from flat_ref_idx
                for pid_idx in range(n_people):
                    wav_idx = flat_ref_idx(vow_idx, pit_idx, pid_idx)
                    wav_idx_shifted = flat_ref_idx(vow_idx, pit_idx + shift_amt, pid_idx)
                    # Add the X-Y (data-label) pair to either the training set or the validation set
                    if (pid_idx != test_pid):
                        X_train_base.extend(all_spectrograms[wav_idx].T)
                        X_train_shifted.extend(simple_ffts_pitch_shift(all_spectrograms[wav_idx], shift_amt).T)
                        Y_train.extend(all_spectrograms[wav_idx_shifted].T)
                    else:
                        X_val_base.extend(all_spectrograms[wav_idx].T)
                        X_val_shifted.extend(simple_ffts_pitch_shift(all_spectrograms[wav_idx], shift_amt).T)
                        Y_val.extend(all_spectrograms[wav_idx_shifted].T)
                        list_of_test_idx.append((wav_idx,wav_idx_shifted))

# Convert to absolute np arrays to be input into NN
X_train_base = np.abs(np.array(X_train_base))
Y_train = np.abs(np.array(Y_train))
X_val_base = np.abs(np.array(X_val_base))
Y_val = np.abs(np.array(Y_val))
X_train_shifted = np.abs(np.array(X_train_shifted))
X_val_shifted = np.abs(np.array(X_val_shifted))
list_of_test_idx = np.array(list_of_test_idx)

# X_train, X_val: (_,1026 = 513*2), (_,1026 = 513*2)
X_train = np.hstack((X_train_base,X_train_shifted))
X_val = np.hstack((X_val_base,X_val_shifted))

In [ ]:
print("X_train_base: {}".format(X_train_base.shape), "\tX_train_shifted: {}".format(X_train_shifted.shape))
print("X_val_base: {}".format(X_val_base.shape), "\tX_val_shifted: {}".format(X_val_shifted.shape))
print("X_train: {}".format(X_train.shape), "\tY_val: {}".format(Y_val.shape))
print("X_val: {}".format(X_val.shape), "\t\tY_train: {}".format(Y_train.shape))

x_axis = librosa.core.fft_frequencies(sample_rate, 1024)
plt.figure(figsize=(10, 5))
plt.plot(x_axis, X_train_base[103000], label='Original', linewidth=1)
plt.plot(x_axis, X_train_shifted[103000], label='Pitch shifted', linewidth=1)
plt.plot(x_axis, Y_train[103000], label='Expected', linewidth=1)
plt.xlim(0, 5000)
plt.xlabel('Frequency')
plt.ylabel('abs(fft)')
plt.legend(loc='best')
plt.title("FFT for data point 103000")